# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
conn = sqlite3.connect("/content/Db-IMDB.db")

#### Overview of all tables

In [5]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn) # Returns DF format
tables = tables["Table_Name"].values.tolist()
tables

['Movie',
 'Genre',
 'Language',
 'Country',
 'Location',
 'M_Location',
 'M_Country',
 'M_Language',
 'M_Genre',
 'Person',
 'M_Producer',
 'M_Director',
 'M_Cast']

In [6]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [ ]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results)
    assert (q1_results.shape == (232,3))

query1 = """ 
SELECT TRIM(m.title) AS Movie_Title,
       CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) as m_year,
       TRIM(p.Name) AS Actor_Name

FROM M_Director as d
  INNER JOIN Movie as m
    ON d.MID = m.MID
  INNER JOIN Person as p
    ON d.PID = p.PID
  INNER JOIN M_Genre as gm
    ON m.MID = gm.MID
  INNER JOIN Genre as g
    ON gm.GID = g.GID

WHERE g.Name LIKE '%comedy%'
  AND (
    m_year%4 = 0
    AND m_year % 100 != 0
    OR  m_year % 400 = 0
  ) 
"""
grader_1(query1)

                           Movie_Title  m_year             Actor_Name
0                           Mastizaade    2016           Milap Zaveri
1    Harold & Kumar Go to White Castle    2004           Danny Leiner
2                   Gangs of Wasseypur    2012         Anurag Kashyap
3          Around the World in 80 Days    2004           Frank Coraci
4               The Accidental Husband    2008          Griffin Dunne
..                                 ...     ...                    ...
227                        Let's Enjoy    2004  Siddharth Anand Kumar
228                            Sathyam    2008        Amma Rajasekhar
229                      Tandoori Love    2008          Oliver Paulus
230                        Le Halua Le    2012            Raja Chanda
231                  Raja Aur Rangeeli    1996       K.S. Prakash Rao

[232 rows x 3 columns]
CPU times: user 62.9 ms, sys: 10.4 ms, total: 73.3 ms
Wall time: 78.6 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [ ]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))


query2 = """ 

SELECT DISTINCT TRIM(P.Name) as Actor_Names

FROM M_Cast as C
  INNER JOIN Movie as M
    ON C.MID = M.MID
  INNER JOIN Person as P
    ON TRIM(C.PID) = TRIM(P.PID)

WHERE M.title = 'Anand';

"""
grader_2(query2)

        Actor_Names
0  Amitabh Bachchan
1     Rajesh Khanna
2     Sumita Sanyal
3        Ramesh Deo
4         Seema Deo
5    Asit Kumar Sen
6        Dev Kishan
7      Atam Prakash
8     Lalita Kumari
9            Savita
CPU times: user 271 ms, sys: 10.4 ms, total: 281 ms
Wall time: 294 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [ ]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 =""" 

SELECT p.PID 
FROM Person AS p 
  INNER JOIN 
  (
    SELECT TRIM(mc.PID) PD, mc.MID from M_cast AS mc 
    WHERE mc.MID 
      IN 
      (
        SELECT mv.MID 
        FROM Movie AS mv 
        WHERE CAST(SUBSTR(mv.year,-4) AS Integer)<1970
      )
  ) AS r1 
    ON r1.PD=p.PID 
"""
query_more_1990 =""" 

SELECT p.PID 
FROM Person p 
  INNER JOIN 
  (
    SELECT TRIM(mc.PID) AS PD, mc.MID 
    FROM M_cast AS mc 
    WHERE mc.MID 
      IN 
        (
          SELECT mv.MID 
          FROM Movie AS mv 
          WHERE CAST(SUBSTR(mv.year,-4) AS Integer)>1990
        )
  ) AS r1 
    ON r1.PD=p.PID """
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question 

(4942, 1)
(62570, 1)
True
CPU times: user 249 ms, sys: 7.76 ms, total: 257 ms
Wall time: 258 ms


In [ ]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """ 
WITH
    query_less_1970 AS 
    (
      SELECT DISTINCT TRIM(MC.PID) AS PID

      FROM Movie AS M
        INNER JOIN M_Cast AS MC
          ON M.MID = MC.MID

      WHERE CAST(SUBSTR(M.year,-4) AS Integer) < 1970
    ),

    query_more_1990 AS 
    (
      SELECT DISTINCT TRIM(MC.PID) as PID

      FROM Movie AS M
        INNER JOIN M_Cast AS MC
          ON M.MID = MC.MID

      WHERE CAST(SUBSTR(M.year,-4) AS Integer) > 1990
    )

SELECT DISTINCT TRIM(P.Name) as Actor_Name

FROM query_less_1970 AS QL
  JOIN query_more_1990 AS QM
    ON QL.PID = QM.PID

  INNER JOIN Person AS P
    ON QL.PID = TRIM(P.PID)

"""
grader_3(query3)

         Actor_Name
0      Rishi Kapoor
1  Amitabh Bachchan
2            Asrani
3      Zohra Sehgal
4   Parikshat Sahni
5     Rakesh Sharma
6       Sanjay Dutt
7         Ric Young
8             Yusuf
9    Suhasini Mulay
CPU times: user 336 ms, sys: 9.08 ms, total: 345 ms
Wall time: 346 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [ ]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

#query_4a =""" *** Write a query, which will return all the directors(id's) along with the number of movies they directed *** """
query_4a =""" 

SELECT
  TRIM(MD.PID) AS D_PID,
  COUNT(*) AS M_Count

FROM
  Movie AS M
  JOIN M_Director AS MD
    ON M.MID = MD.MID

GROUP BY
  D_PID

"""

print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

       D_PID  M_Count
0  nm0000180        1
1  nm0000187        1
2  nm0000229        1
3  nm0000269        1
4  nm0000386        1
5  nm0000487        2
6  nm0000965        1
7  nm0001060        1
8  nm0001162        1
9  nm0001241        1
True
CPU times: user 14.1 ms, sys: 0 ns, total: 14.1 ms
Wall time: 17.2 ms


In [ ]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    print(q4_results.shape)
    assert (q4_results.shape == (58,2))

query4 = f"""
WITH
  Director_movie_count AS
  (
    {query_4a}
  )

SELECT
  DISTINCT P.Name AS Directr_Name,
  R.M_Count AS Movie_Count

FROM Person AS P
  INNER JOIN Director_movie_count AS R
    ON P.PID = R.D_PID

WHERE
  Movie_Count >= 10

ORDER BY
  Movie_Count DESC
"""
grader_4(query4)

            Directr_Name  Movie_Count
0           David Dhawan           39
1           Mahesh Bhatt           35
2        Ram Gopal Varma           30
3           Priyadarshan           30
4           Vikram Bhatt           29
5   Hrishikesh Mukherjee           27
6            Yash Chopra           21
7         Shakti Samanta           19
8        Basu Chatterjee           19
9           Subhash Ghai           18
(58, 2)
CPU times: user 23.7 ms, sys: 0 ns, total: 23.7 ms
Wall time: 26.3 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [ ]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10))
    print(query_5aa.shape)
    return (query_5aa.shape == (8846,3))

#query_5aa =""" *** Write your query that will get movie id, and number of people for each gender *** """
query_5aa =""" 
SELECT
  TRIM(M.MID) AS Movie_ID,
  P.Gender as Gender,
  COUNT(*) as Number_of_Movie

FROM
  Movie AS M
  INNER JOIN M_Cast AS MC
    ON M.MID = MC.MID
  INNER JOIN Person AS P
    ON TRIM(MC.PID) = P.PID

GROUP BY
  Movie_ID, Gender

ORDER BY
  Movie_ID

"""

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10))
    print(query_5ab.shape) 
    return (query_5ab.shape == (3469, 3))

#query_5ab =""" *** Write your query that will have at least one male actor try to use query that you have written above *** """
query_5ab =f"""
WITH
  Movie_gend_count AS
  (
    {query_5aa}
  )

SELECT *

FROM
  Movie_gend_count AS MG

WHERE
  Trim(MG.Gender) = 'Male'

"""

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

    Movie_ID  Gender  Number_of_Movie
0  tt0021594    None                1
1  tt0021594  Female                3
2  tt0021594    Male                5
3  tt0026274    None                2
4  tt0026274  Female               11
5  tt0026274    Male                9
6  tt0027256    None                2
7  tt0027256  Female                5
8  tt0027256    Male                8
9  tt0028217  Female                3
(8846, 3)
True
    Movie_ID Gender  Number_of_Movie
0  tt0021594   Male                5
1  tt0026274   Male                9
2  tt0027256   Male                8
3  tt0028217   Male                7
4  tt0031580   Male               27
5  tt0033616   Male               46
6  tt0036077   Male               11
7  tt0038491   Male                7
8  tt0039654   Male                6
9  tt0040067   Male               10
(3469, 3)
True
CPU times: user 617 ms, sys: 19.9 ms, total: 637 ms
Wall time: 635 ms


In [ ]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    print(q5a_results.shape)
    assert (q5a_results.shape == (4,2))

query5a = f""" 
WITH
  Movie_male_count AS
    (
      {query_5ab}
    )

SELECT
  CAST(SUBSTR(TRIM(M.year),-4) AS INTEGER) as Movie_Year,
  COUNT(*) AS Female_Cast_Only_Movies

FROM
  Movie AS M

WHERE
  M.MID 
  NOT IN 
    ( 
      SELECT Movie_ID 
      FROM Movie_male_count 
    )

GROUP BY
  Year

"""
grader_5a(query5a)

   Movie_Year  Female_Cast_Only_Movies
0        1939                        1
1        1999                        1
2        2000                        1
3        2018                        1
(4, 2)
CPU times: user 323 ms, sys: 12.9 ms, total: 336 ms
Wall time: 392 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [ ]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

# 1) calculate total number of movies in each year

query5b = f""" 
WITH 
  Female_Only_Movies AS
    (
      {query5a}
    ),

  Total_movies_yearly AS
  (
    SELECT
      CAST(SUBSTR(TRIM(M.year),-4) AS INTEGER) as m_Year,
      COUNT(*) AS Total_Movies

    FROM
      Movie AS M

    GROUP BY
      m_Year
  )

SELECT
  F.Movie_Year,
  (F.Female_Cast_Only_Movies/CAST(T.Total_Movies AS FLOAT)) AS Percentage_Female_Only_Movie,
  T.Total_Movies

FROM
  Female_Only_Movies AS F
  LEFT JOIN Total_movies_yearly AS T
    ON F.Movie_Year = T.m_Year

"""
grader_5b(query5b)

   Movie_Year  Percentage_Female_Only_Movie  Total_Movies
0        1939                      0.500000             2
1        1999                      0.015152            66
2        2000                      0.015625            64
3        2018                      0.009615           104
CPU times: user 482 ms, sys: 25.2 ms, total: 507 ms
Wall time: 595 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [ ]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    print(q6_results.shape)
    assert (q6_results.shape == (3473, 2))

query6 = """

SELECT
  M.title AS Movie_Title,
  COUNT(DISTINCT TRIM(P.Name)) AS Cast_Size

FROM
  Movie AS M
  INNER JOIN
    M_Cast AS MC
    ON M.MID = TRIM(MC.MID)   
  INNER JOIN
    Person as P
    ON TRIM(MC.PID) = P.PID

GROUP BY
  M.MID

ORDER BY
  Cast_Size DESC

"""
grader_6(query6)

                  Movie_Title  Cast_Size
0               Ocean's Eight        238
1                    Apaharan        233
2                        Gold        214
3             My Name Is Khan        213
4  Captain America: Civil War        191
5                    Geostorm        170
6                     Striker        165
7                        2012        154
8                      Pixels        144
9       Yamla Pagla Deewana 2        140
(3473, 2)
CPU times: user 447 ms, sys: 14.7 ms, total: 461 ms
Wall time: 540 ms


## Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D

In [ ]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    print(q7a_results.shape)
    assert (q7a_results.shape == (78, 2))

query7a = """
SELECT
  CAST(SUBSTR(TRIM(M.year),-4) AS INTEGER) as Movie_Year,
  COUNT(*) AS Total_Movies

FROM
  Movie AS M

GROUP BY
  Movie_Year
"""
grader_7a(query7a)

# using the above query, you can write the answer to the given question

   Movie_Year  Total_Movies
0        1931             1
1        1936             3
2        1939             2
3        1941             1
4        1943             1
5        1946             2
6        1947             2
7        1948             3
8        1949             3
9        1950             2
(78, 2)
CPU times: user 9.06 ms, sys: 0 ns, total: 9.06 ms
Wall time: 9.49 ms


In [ ]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

"""   
    *** 
    Write a query that will do joining of the above table(7a) with itself 
    such that you will join with only rows if the second tables year is <= current_year+9 and more than or equal current_year
    *** 
          """

query7b = f"""
WITH
 first_table AS
 (
   {query7a}
 ),
 second_table AS
 (
   {query7a}
 )
SELECT
  T1.Movie_Year AS T1_Movie_Year,
  T1.Total_Movies AS T1_Total_Movies,
  T2.Movie_Year AS T2_Movie_Year,
  T2.Total_Movies AS T2_Total_Movies

FROM first_table AS T1
  JOIN second_table AS T2
    ON T2.Movie_Year <= T1.Movie_Year+9 
    AND T2.Movie_Year>=T1.Movie_Year
"""
grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

   T1_Movie_Year  T1_Total_Movies  T2_Movie_Year  T2_Total_Movies
0           1931                1           1931                1
1           1931                1           1936                3
2           1931                1           1939                2
3           1936                3           1936                3
4           1936                3           1939                2
5           1936                3           1941                1
6           1936                3           1943                1
7           1939                2           1939                2
8           1939                2           1941                1
9           1939                2           1943                1
CPU times: user 16.2 ms, sys: 0 ns, total: 16.2 ms
Wall time: 16.7 ms


In [ ]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = f"""
WITH
  Joined_tables AS
  (
    {query7b}
  ),
  Decade_table AS
  (
    SELECT
      SUM(T2_Total_Movies) AS Decade_Count,
      T1_Movie_Year AS Decade

    FROM
      Joined_tables

    GROUP BY
      T1_Movie_Year
  )

SELECT
  MAX(Decade_Count) AS Decade_Movie_Count , Decade
FROM Decade_table
"""
grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

   Decade_Movie_Count  Decade
0                1203    2008
CPU times: user 12.2 ms, sys: 0 ns, total: 12.2 ms
Wall time: 13.1 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [ ]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

#query8a = """ *** Write a query that will results in number of movies actor-director worked together ***"""
query8a = """
SELECT DISTINCT MC.PID AS Actor,
        MD.PID AS Director,
        count(*) AS Movie_count

FROM MOVIE AS M
  INNER JOIN
    M_Cast AS MC
    ON M.MID = MC.MID
  INNER JOIN M_Director AS MD
    ON M.MID = MD.MID

GROUP BY
  MC.PID, MD.PID
"""
grader_8a(query8a)

# using the above query, you can write the answer to the given question

        Actor   Director  Movie_count
0   nm0000002  nm0496746            1
1   nm0000027  nm0000180            1
2   nm0000039  nm0896533            1
3   nm0000042  nm0896533            1
4   nm0000047  nm0004292            1
5   nm0000073  nm0485943            1
6   nm0000076  nm0000229            1
7   nm0000092  nm0178997            1
8   nm0000093  nm0000269            1
9   nm0000096  nm0113819            1
CPU times: user 741 ms, sys: 25.2 ms, total: 766 ms
Wall time: 886 ms


In [ ]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = f""" 
WITH Act_Dir_count AS
(
  {query8a}
)

SELECT ADC.Actor, ADC.Movie_count
FROM Act_Dir_count AS ADC
WHERE (ADC.Actor, ADC.Movie_count) IN
(
  SELECT ADC1.Actor, max(ADC1.Movie_count)
  FROM Act_Dir_count AS ADC1
  GROUP BY ADC1.Actor
)

AND ADC.Director = (
  SELECT P.PID
  FROM Person AS P
  WHERE P.Name LIKE '%Yash Chopra%'
)
"""
grader_8(query8)

        Actor  Movie_count
0   nm0004434            7
1   nm0007181            2
2   nm0015296            1
3   nm0019463            1
4   nm0046230            1
5   nm0052570            1
6   nm0080266            1
7   nm0080385            1
8   nm0081070            1
9   nm0085944            1
(245, 2)
CPU times: user 1.28 s, sys: 46.4 ms, total: 1.32 s
Wall time: 1.57 s


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [ ]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """
WITH 
    SRK_PID AS
    (
        SELECT
            TRIM(P.PID) AS PID
        FROM
            Person AS P
        WHERE
            Trim(P.Name) like '%Shah Rukh Khan%'
    ),
    SRK_Movies AS
    (
        SELECT
            DISTINCT TRIM(MC.MID) AS MID,
            S.PID
        FROM
            M_Cast AS MC,
            SRK_PID AS S
        WHERE
            TRIM(MC.PID) = S.PID
    )
      SELECT
            DISTINCT TRIM(MC.PID) AS S1_PID
        FROM
            M_Cast MC,
            SRK_Movies AS SM
        WHERE
            TRIM(MC.MID) = SM.MID AND
            S1_PID != SM.PID

"""
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

      S1_PID
0  nm0004418
1  nm1995953
2  nm2778261
3  nm0631373
4  nm0241935
5  nm0792116
6  nm1300111
7  nm0196375
8  nm1464837
9  nm2868019
(2382, 1)
CPU times: user 106 ms, sys: 5.22 ms, total: 111 ms
Wall time: 184 ms


In [ ]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = f""" 
WITH 
    SRK_1_PID AS
    (
      {query9a}
    ),

    SRK_2_movies AS
    (
      SELECT
          DISTINCT TRIM(MC.MID) AS S2_MID
      FROM
          M_Cast AS MC,
          SRK_1_PID
      WHERE
          TRIM(MC.PID) = SRK_1_PID.S1_PID
    ),

    SRK_12_PID AS 
    (
      SELECT
          DISTINCT TRIM(MC.PID) AS S12_PID
      FROM
          M_Cast AS MC,
          SRK_2_movies AS S2M
      WHERE
          TRIM(MC.MID) = S2M.S2_MID
    ),

    S2_Actor AS
    (
      SELECT 
          DISTINCT S12.S12_PID AS S2_PID
      FROM
          SRK_12_PID AS S12
      WHERE
          S2_PID NOT IN ( SELECT S1_PID FROM SRK_1_PID )
    )

SELECT
    P.Name AS Actor_Name
FROM
    Person AS P,
    S2_Actor
WHERE
    P.PID = S2_Actor.S2_PID
    AND P.PID != (
      SELECT P.PID
      FROM Person AS P
      WHERE P.Name LIKE '%Shah Rukh Khan%'
    )

"""
grader_9(query9)

               Actor_Name
0            Freida Pinto
1             Rohan Chand
2            Damian Young
3         Waris Ahluwalia
4   Caroline Christl Long
5           Rajeev Pahuja
6       Michelle Santiago
7         Alicia Vikander
8            Dominic West
9          Walton Goggins
(25698, 1)
CPU times: user 405 ms, sys: 7.98 ms, total: 413 ms
Wall time: 416 ms
